Мы будем использовать алгоритм PageRank (разреженная цепь Маркова с телепортацией, α=0.85), чтобы для каждого трёхсловного паттерна (subj_pred_obj):

Найти стационарную вероятность π — насколько часто паттерн «встречается» в устоявшейся модели.

Посчитать энтропийную скорость H — среднее число бит неопределённости при переходе от одного паттерна к следующему.

In [9]:
import pandas as pd
import networkx as nx
import numpy as np

def build_graph_from_sequences(seqs):
    G = nx.DiGraph()
    for seq in seqs:
        for u, v in zip(seq, seq[1:]):
            if G.has_edge(u, v):
                G[u][v]['weight'] += 1
            else:
                G.add_edge(u, v, weight=1)
    return G

def pagerank_and_entropy(G, alpha=0.85):
    # 1) Считаем PageRank
    pr = nx.pagerank(G, alpha=alpha, weight='weight', tol=1e-6)
    pi_series = pd.Series(pr, name='pi').sort_values(ascending=False)

    # 2) Считаем entropy rate
    H = 0.0
    for i in G.nodes():
        pi_i = pr.get(i, 0.0)
        nbrs = G[i]  # это dict: {j: attr_dict, ...}
        total = sum(attr['weight'] for attr in nbrs.values())
        if total == 0:
            continue
        for j, attr in nbrs.items():
            p_ij = attr['weight'] / total
            H -= pi_i * p_ij * np.log2(p_ij)
    return pi_series, H


if __name__ == "__main__":
    # 1) Загрузка и подготовка, как у вас
    jokes_df = (pd.read_csv('output_files/jokes_relations_clean.csv')
                  .dropna(subset=['subj','pred','obj']))
    lit_df   = (pd.read_csv('output_files/lit_relations_lit_clean.csv')
                  .dropna(subset=['subj','pred','obj']))

    jokes_df['order'] = jokes_df.groupby('joke_id').cumcount()
    lit_df['order']   = lit_df.groupby('segment_id').cumcount()

    joke_seqs = (
        jokes_df
          .sort_values(['joke_id','order'])
          .groupby('joke_id')[['subj','pred','obj']]
          .apply(lambda g: (g['subj'] + '_' + g['pred'] + '_' + g['obj']).tolist())
          .tolist()
    )
    lit_seqs = (
        lit_df
          .sort_values(['segment_id','order'])
          .groupby('segment_id')[['subj','pred','obj']]
          .apply(lambda g: (g['subj'] + '_' + g['pred'] + '_' + g['obj']).tolist())
          .tolist()
    )

    # 2) Строим графы
    G_joke = build_graph_from_sequences(joke_seqs)
    G_lit  = build_graph_from_sequences(lit_seqs)

    # 3) Запускаем PageRank + считаем энтропию
    pi_joke, H_joke = pagerank_and_entropy(G_joke, alpha=0.85)
    pi_lit,  H_lit  = pagerank_and_entropy(G_lit,  alpha=0.85)

    # 4) Сохраняем и выводим
    pi_joke.to_csv('mark/pi_joke_stationary.csv', encoding='utf-8')
    pi_lit.to_csv( 'mark/pi_lit_stationary.csv',  encoding='utf-8')
    with open('entropy_rates.txt','w',encoding='utf-8') as f:
        f.write(f"Entropy rate (jokes):      {H_joke:.6f} bit/step\n")
        f.write(f"Entropy rate (literature): {H_lit:.6f} bit/step\n")

    print(f"Entropy rate (jokes):      {H_joke:.6f} bit/step")
    print(f"Entropy rate (literature): {H_lit:.6f} bit/step")


Entropy rate (jokes):      0.130890 bit/step
Entropy rate (literature): 0.084958 bit/step


**Энтропийная скорость H**

- **Большое H** (например, ~0.13 в шутках) означает, что переходы между паттернами менее предсказуемы и более разнообразны.  
- **Малое H** (например, ~0.085 в литературе) отражает более упорядоченные, повторяющиеся сочетания.  

**Шутки**  
Высокая энтропия говорит о том, что шутки часто «прыгают» между разными паттернами: неожиданные комбинации делают текст смешным. Такое свойство можно использовать как признак для автоматического выявления шуток среди обычного текста.

**Литературные тексты**  
Низкая энтропия характеризует более стройный, последовательный стиль: авторы придерживаются устойчивых фразовых оборотов. Это помогает отличать прозу или поэзию от юмористических жанров.

In [11]:
# … после вызова pagerank_and_entropy …

# Выводим топ-20 самых «центральных» паттернов
print("Top-20 паттернов в шутках (π_joke):")
print(pi_joke.head(20))

print("\nTop-20 паттернов в литературе (π_lit):")
print(pi_lit.head(20))


Top-20 паттернов в шутках (π_joke):
звонок_в_дверь          0.000303
раз_в_день              0.000226
раз_в_год               0.000180
раз_в_неделя            0.000176
раз_в_жизнь             0.000161
стук_в_дверь            0.000147
сборная_по_футбол       0.000132
ответ_на_вопрос         0.000126
раз_в_месяц             0.000122
чемпионат_по_футбол     0.000118
жена_говорить_муж       0.000116
вовочка_тянуть_рука     0.000105
муж_говорить_жена       0.000104
голос_из_зал            0.000102
объявление_в_газета     0.000085
борьба_с_коррупция      0.000074
тест_на_беременность    0.000067
цена_на_бензин          0.000064
жена_открывать_дверь    0.000059
фингал_под_глаз         0.000059
Name: pi, dtype: float64

Top-20 паттернов в литературе (π_lit):
раз_в_жизнь            0.000109
утро_до_вечер          0.000108
день_и_ночь            0.000095
рука_и_нога            0.000084
раз_в_день             0.000074
рубль_в_месяц          0.000068
ответ_на_вопрос        0.000062
отец_и_мать    